## Descripción del dataset


 En este dataset se desea pronosticar el precio de vhiculos usados. El dataset
 original contiene las siguientes columnas:

 - Car_Name: Nombre del vehiculo.
 - Year: Año de fabricación.
 - Selling_Price: Precio de venta.
 - Present_Price: Precio actual.
 - Driven_Kms: Kilometraje recorrido.
 - Fuel_type: Tipo de combustible.
 - Selling_Type: Tipo de vendedor.
 - Transmission: Tipo de transmisión.
 - Owner: Número de propietarios.

 El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
 en la carpeta "files/input/".

 Los pasos que debe seguir para la construcción de un modelo de
 pronostico están descritos a continuación.



 ## Paso 1.
 Preprocese los datos.
 - Cree la columna 'Age' a partir de la columna 'Year'.
   Asuma que el año actual es 2021.
 - Elimine las columnas 'Year' y 'Car_Name'.



## Paso 2.
 Divida los datasets en x_train, y_train, x_test, y_test.

In [322]:
import pandas as pd 
import os
import pickle
import gzip
import json
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif,f_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression

def carga_y_limpieza(train_path, test_path):
    # Leer los datasets descomprimidos
    train_dataset = pd.read_csv(train_path)
    test_dataset = pd.read_csv(test_path)
    print(train_dataset.shape)
    print(test_dataset.shape)

    # -- Elimine los registros con informacion no disponible.
    train_dataset.dropna(inplace=True)
    test_dataset.dropna(inplace=True)

    train_dataset["Age"] = 2021 - train_dataset["Year"]
    train_dataset = train_dataset.drop(["Car_Name", "Year"], axis=1)
    x_train = train_dataset.drop(columns=["Present_Price"])
    y_train = train_dataset["Present_Price"]


    test_dataset["Age"] = 2021 - test_dataset["Year"]
    test_dataset = test_dataset.drop(["Car_Name", "Year"], axis=1)
    x_test = test_dataset.drop(columns=["Present_Price"])
    y_test = test_dataset["Present_Price"]
    categorical_columns=["Fuel_Type","Selling_type","Transmission"]
    numerical_columns=["Selling_Price","Driven_kms","Owner","Age"]

    print(x_train.shape)
    print(x_test.shape)

    return x_train, y_train, x_test, y_test, categorical_columns,numerical_columns


## Paso 3.
Cree un pipeline para el modelo de clasificación. Este pipeline debe
contener las siguientes capas:
- Transforma las variables categoricas usando el método
  one-hot-encoding.
- Escala las variables numéricas al intervalo [0, 1].
- Selecciona las K mejores entradas.
- Ajusta un modelo de regresion lineal.

In [323]:
def build_pipeline(estimator, categorical_columns=None, numeric_columns=None):
    # --- Preprocesamiento ---
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
            ('num', MinMaxScaler(), numeric_columns)
        ]
    )

    # --- Pipeline completo ---
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),                      
        ("selectkbest", SelectKBest()),  # selección de variables
        ("estimator", estimator)                  
    ])

    return pipeline

## Paso 4.
Optimice los hiperparametros del pipeline usando validación cruzada.
Use 10 splits para la validación cruzada. Use el error medio absoluto
para medir el desempeño modelo.


In [324]:
def make_grid_search(estimator, param_grid, cv=10):

    from sklearn.model_selection import GridSearchCV

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        scoring=make_scorer(mean_absolute_error, greater_is_better=False),
        cv=cv,
        n_jobs=-1,
        refit=True,
    )

    return grid_search 

## Paso 5.
 Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
 Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.


In [325]:
def save_estimator(estimator):

    os.makedirs("../files/models", exist_ok=True)
    with gzip.open("../files/models/model.pkl.gz", "wb") as f:
        pickle.dump(estimator, f)

## Paso 6.
 Calcule las metricas r2, error cuadratico medio, y error absoluto medio
 para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
 files/output/metrics.json. Cada fila del archivo es un diccionario con
 las metricas de un modelo. Este diccionario tiene un campo para indicar
 si es el conjunto de entrenamiento o prueba. Por ejemplo:

 {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
 {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}


In [326]:
def guardar_metricas(best_model,y_train, y_train_pred, y_test, y_test_pred, output_path):
    # Calcular métricas para el conjunto de entrenamiento
    train_metrics = {
        'type': 'metrics',
        'dataset': 'train',
        'r2': r2_score(y_train, y_train_pred),
        'mse': mean_squared_error(y_train, y_train_pred),
        'mad': mean_absolute_error(y_train, y_train_pred)
    }

    # Calcular métricas para el conjunto de prueba
    test_metrics = {
        'type': 'metrics',
        'dataset': 'test',
        'r2': r2_score(y_test, y_test_pred),
        'mse': mean_squared_error(y_test, y_test_pred),
        'mad': mean_absolute_error(y_test, y_test_pred)
    }

    # Guardar las métricas en un archivo JSON
    with open(output_path, 'w') as f:
        f.write(json.dumps(train_metrics) + '\n')
        f.write(json.dumps(test_metrics) + '\n')

## Ejecución

In [327]:
def regresion_lineal():
    train_path="../files/input/train_data.csv"
    test_path="../files/input/test_data.csv"

    # --- Paso 1 y 2: Carga y limpieza ---
    x_train, y_train, x_test, y_test, categorical_columns, numeric_columns = carga_y_limpieza(train_path, test_path)

        # --- Paso 3: Pipeline ---
    pipeline = build_pipeline(
        estimator=LinearRegression(),
        categorical_columns=categorical_columns,
        numeric_columns=numeric_columns,
    )

    param_grid = {
    'selectkbest__k': [i for i in range(1, 10)],  # Probar con todas las características
    'selectkbest__score_func': [f_regression, mutual_info_regression], 
}
    
    grid_search = make_grid_search(
        estimator=pipeline,
        param_grid=param_grid,
        cv=10,
    )

    grid_search.fit(x_train,y_train)

    ##Guardar estimador
    best_model = grid_search.best_estimator_

    save_estimator(grid_search)
    y_train_pred = best_model.predict(x_train)
    y_test_pred = best_model.predict(x_test)
    

    ##Guardar metricas

    output_path = "../files/output/metrics.json"
    guardar_metricas(best_model, y_train, y_train_pred, y_test, y_test_pred, output_path)

regresion_lineal()


(211, 9)
(90, 9)
(211, 7)
(90, 7)
